# Sequence Modelling using bidirectional GRU and Attention Layer

## References

__DATA__

1. https://www.kaggle.com/ultrajack/modern-renaissance-poetry/data

__Code__

1. https://github.com/joosthub/PyTorchNLPBook/blob/master/chapters/chapter_5/5_2_CBOW/5_2_munging_frankenstein.ipynb

In [ ]:
import numpy as np
from collections import Counter
import string
import torch
from torch.utils.data import Dataset
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
from argparse import Namespace
import torch.optim as optim
from tqdm.notebook import tqdm
import torch.nn.functional as F
import re
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset Build up


In [ ]:
poet_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/PoetryGenerator/data/poetryfoundation.csv")
poet_df.head()

,author,content,poem name,age,type
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore


In [ ]:
poet_df['author'].unique()

array(['WILLIAM SHAKESPEARE', 'DUCHESS OF NEWCASTLE MARGARET CAVENDISH',
       'THOMAS BASTARD', 'EDMUND SPENSER', 'RICHARD BARNFIELD',
       'SIR WALTER RALEGH', 'QUEEN ELIZABETH I', 'JOHN DONNE',
       'JOHN SKELTON', 'CHRISTOPHER MARLOWE', 'LADY MARY WROTH',
       'ROBERT SOUTHWELL, SJ', 'WILLIAM BYRD', 'GEORGE GASCOIGNE',
       'HENRY VIII, KING OF ENGLAND', 'SIR THOMAS WYATT', 'EN JONSON',
       'ORLANDO GIBBONS', 'THOMAS NASHE', 'SIR PHILIP SIDNEY',
       'SECOND BARON VAUX OF HARROWDEN THOMAS, LORD VAUX',
       'HENRY HOWARD, EARL OF SURREY', 'GEORGE CHAPMAN', 'THOMAS CAMPION',
       'ISABELLA WHITNEY', 'SAMUEL DANIEL', 'THOMAS HEYWOOD',
       'GIOVANNI BATTISTA GUARINI', 'SIR EDWARD DYER', 'THOMAS LODGE',
       'JOHN FLETCHER', 'EDGAR LEE MASTERS', 'WILLIAM BUTLER YEATS',
       'FORD MADOX FORD', 'IVOR GURNEY', 'CARL SANDBURG', 'EZRA POUND',
       'ELINOR WYLIE', 'GEORGE SANTAYANA', 'LOUISE BOGAN',
       'KENNETH SLESSOR', 'HART CRANE', 'D. H. LAWRENCE',
       'H

### import NLTK and Tokenizing

In [ ]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

poet_dataset = {'first': [], 'second': []}

num_poets = 500

for i in range(num_poets):
    sentences = tokenizer.tokenize(poet_df['content'][i])
    sentences_len = len(sentences)
    WINDOW_SIZE = 2
    for index in range(sentences_len - WINDOW_SIZE):
        prev_sentence = sentences[index]
        next_sentence = sentences[index + 1]

        # print("1 " + prev_sentence)
        # print("2 " + next_sentence)
        poet_dataset['first'].append(prev_sentence)
        poet_dataset['second'].append(next_sentence)

poet_dataset = pd.DataFrame.from_dict(poet_dataset)

In [ ]:
poet_dataset.head()

,first,second
0,Let the bird of loudest lay\r\nOn the sole Ara...,"But thou shrieking harbinger,\r\nFoul precurre..."
1,"But thou shrieking harbinger,\r\nFoul precurre...",From this session interdict\r\nEvery fowl of t...
2,From this session interdict\r\nEvery fowl of t...,"Let the priest in surplice white,\r\nThat defu..."
3,"Let the priest in surplice white,\r\nThat defu...","And thou treble-dated crow,\r\nThat thy sable ..."
4,"And thou treble-dated crow,\r\nThat thy sable ...",Here the anthem doth commence:\r\nLove and con...


### Document Cleansing and Formating Train Data

the train data would be

prev_sentence | next_sentence

In [ ]:
import re
import string

def preprocess_text(text):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

def preprocess_row(row):
    row['first'] = preprocess_text(row['first'])
    row['second'] = preprocess_text(row['second'])
    return row

In [ ]:
poet_dataset = poet_dataset.apply(lambda row: preprocess_row(row), axis=1)

In [ ]:
poet_dataset.head()

,first,second
0,let the bird of loudest lay on the sole arabia...,"but thou shrieking harbinger , foul precurrer ..."
1,"but thou shrieking harbinger , foul precurrer ...",from this session interdict every fowl of tyra...
2,from this session interdict every fowl of tyra...,"let the priest in surplice white , that defunc..."
3,"let the priest in surplice white , that defunc...","and thou treble dated crow , that thy sable ge..."
4,"and thou treble dated crow , that thy sable ge...",here the anthem doth commence love and constan...


In [ ]:
# splitting data into
# train val and test

poet_dataset['split'] = 'train'
def assign_label(row):
    magic_number = np.random.randint(0, 10)
    if magic_number > 6:
        valortest = np.random.randint(0, 2)
        if valortest == 0:
            return 'val'
        elif valortest == 1:
            return 'test'
    else:
        return 'train'
           
poet_dataset['split'] = poet_dataset.apply(lambda row: assign_label(row['split']), axis=1)

In [ ]:
poet_dataset

,first,second,split
0,let the bird of loudest lay on the sole arabia...,"but thou shrieking harbinger , foul precurrer ...",val
1,"but thou shrieking harbinger , foul precurrer ...",from this session interdict every fowl of tyra...,train
2,from this session interdict every fowl of tyra...,"let the priest in surplice white , that defunc...",val
3,"let the priest in surplice white , that defunc...","and thou treble dated crow , that thy sable ge...",train
4,"and thou treble dated crow , that thy sable ge...",here the anthem doth commence love and constan...,train
...,...,...,...
2601,why should i blame her that she filled my days...,what could have made her peaceful with a mind ...,train
2602,what could have made her peaceful with a mind ...,"why , what could she have done , being what sh...",train
2603,"shy one , shy one , shy one of my heart , she ...","she carries in the dishes , and lays them in a...",train
2604,"she carries in the dishes , and lays them in a...",to an isle in the water with her would i go .,train


In [ ]:
poet_dataset['split'].unique()

array(['val', 'train', 'test'], dtype=object)

In [ ]:
poet_dataset.to_csv("/content/drive/My Drive/Colab Notebooks/PoetryGenerator/data/poet_dataframe_sentences.csv", index=False)

## Sequence Vocab

In [ ]:
class SequenceVocabulary(object):
    """Class to extract and process vocabularies for mapping"""
    
    def __init__(self, token_to_idx=None, mask_token="<MASK>", add_unk=True, unk_token="<UNK>"):
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        
        self._idx_to_token = {
            idx: token for token, idx in self._token_to_idx.items()
        }
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        self._mask_token = mask_token
        
        # add begin and end sequence token
        self._begin_of_seq_token = "<BEGIN-OF-SEQUENCE>"
        self._end_of_seq_token = "<END-OF-SEQUENCE>"
        
        self.begin_seq_index = self.add_token(self._begin_of_seq_token)
        self.end_seq_index = self.add_token(self._end_of_seq_token)

        self.mask_index = self.add_token(mask_token)
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
            
    def to_serializeable(self):
        """return a serializeable dictionary"""
        return {
            'token_to_idx': self._token_to_idx,
            'mask_token': self._mask_token,
            'add_unk': self._add_unk,
            'unk_token': self._unk_token
        }
    
    @classmethod
    def from_serializeable(cls, contents):
        """create vocabulary object from serialize dictionary"""
        return cls(**contents)
    
    def add_token(self, token):
        """Add a token and return it's index"""
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        """get the index of a token 
        if not exist returns the unk_index"""
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
        
    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError("the index %d is not in the vocabulary" % index)
        return self._idx_to_token[index]
    
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    
    def __len__(self):
        return len(self._token_to_idx)

## Vectorizer

In [ ]:
class NMTVectorizer(object):
    def __init__(self, source_vocab, target_vocab,
                max_source_length, max_target_length):
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        
    @classmethod
    def from_dataframe(cls, lang_df):
        source_vocab = SequenceVocabulary()
        target_vocab = SequenceVocabulary()
        
        max_source_length, max_target_length = 0, 0
        
        for _, rows in lang_df.iterrows():
            # source
            source_token = rows["first"].split(" ")
            if len(source_token) > max_source_length:
                max_source_length = len(source_token)
            for token in source_token:
                source_vocab.add_token(token)
            
            # target
            target_token = rows["second"].split(" ")
            if len(target_token) > max_target_length:
                max_target_length = len(target_token)
            for token in target_token:
                target_vocab.add_token(token)
                
        return cls(source_vocab, target_vocab,
                  max_source_length, max_target_length)
    
    def _vectorize(self, indices, vector_length=-1, mask_index=0):
        if vector_length < 0:
            vector_length = len(indices)
        
        vector = np.zeros(vector_length, dtype=np.int64)
        vector[:len(indices)] = indices
        vector[len(indices):] = mask_index
        return vector
    
    def _get_source_indices(self, source_text):
        """
        Source indices adding begin_seq_index and
        end_seq_index
        """
        indices = [self.source_vocab.begin_seq_index]
        indices.extend(self.source_vocab.lookup_token(token) for token in
                       source_text.split(" "))
        indices.append(self.source_vocab.end_seq_index)
        
        return indices
    
    def _get_target_indices(self, target_text):
        indices = [self.target_vocab.lookup_token(token)
                   for token in target_text.split(" ")]
        
        x_indices = [self.target_vocab.begin_seq_index] + indices
        y_indices = indices + [self.target_vocab.end_seq_index]
        
        return x_indices, y_indices
    
    def vectorize(self, source_text, target_text, use_dataset_max_length=True):
        source_length = -1
        target_length = -1
        
        if use_dataset_max_length:
            source_length = self.max_source_length + 2
            target_length = self.max_target_length + 1
        
        source_indices = self._get_source_indices(source_text)
        source_vector = self._vectorize(source_indices,
                                       source_length,
                                       mask_index= self.source_vocab.mask_index)
        
        target_x_indices, target_y_indices = self._get_target_indices(target_text)
        
        target_x_vector = self._vectorize(target_x_indices,
                                         target_length,
                                         self.target_vocab.mask_index)
        target_y_vector = self._vectorize(target_y_indices,
                                         target_length,
                                         self.target_vocab.mask_index)
        return {"source_vector": source_vector,
                "target_x_vector": target_x_vector,
                "target_y_vector": target_y_vector,
                "source_length": len(source_indices)}

## Dataset

In [ ]:
class NMTDataset(Dataset):
    def __init__(self, text_df, vectorizer):
        self.text_df = text_df
        self._vectorizer = vectorizer

        self.train_df = self.text_df[self.text_df.split == 'train']
        self.train_size = len(self.train_df)
        
        self.val_df = self.text_df[self.text_df.split == 'val']
        self.val_size = len(self.val_df)
        
        self.test_df = self.text_df[self.text_df.split == 'test']
        self.test_size = len(self.test_df)
        
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                            'val': (self.val_df, self.val_size),
                            'test': (self.test_df, self.test_size)}
        
        self.set_split('train')
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, text_csv):
        """Load dataset from csv and returns the dataset object
        and vectorizer"""
        text_df = pd.read_csv(text_csv)
        train_text_df = text_df[text_df.split == 'train']
        return cls(text_df,
                   NMTVectorizer.from_dataframe(train_text_df))
    
    def get_vectorizer(self):
        """Get vectorizer"""
        return self._vectorizer
    
    def set_split(self, split='train'):
        """Set the split from data"""
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
        
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
            index (int): the index to the data point
        Returns:
            a dict of the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        
        data_dict = self._vectorizer.vectorize(row['first'],
                                               row['second'])
        
        return {
            'x_source': data_dict["source_vector"],
            'x_target': data_dict["target_x_vector"],
            'y_target': data_dict["target_y_vector"],
            'x_source_length': data_dict["source_length"]
        }
    
    def get_num_batches(self, batch_size):
        """Given the batch size return the number of batches in the dataset"""
        return len(self) // batch_size


def generate_nmt_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    """
    Batch Generator
    """
    dataloader = DataLoader(dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last= drop_last)
    
    for data_dict in dataloader:
        lengths = data_dict['x_source_length'].numpy()
        sorted_length_indices = lengths.argsort()[::-1].tolist()
        
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name][sorted_length_indices].to(device)
        
        yield out_data_dict

## Encoder

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class NMTEncoder(nn.Module):
    def __init__(self, num_embeddings, embedding_size, rnn_hidden_size):
        """
        Encoder Module
        Args:
            num_embeddings(int): size of input dimension
            embedding_size(int): embedding dimension
            rnn_hidden_size(int): rnn hidden weight size
        """
        super(NMTEncoder, self).__init__()
        
        self.source_embedding = nn.Embedding(num_embeddings, embedding_size, padding_idx=0)
        self.birnn = nn.GRU(embedding_size, rnn_hidden_size, bidirectional=True, batch_first=True)
        
    def forward(self, x_source, x_lengths):
        """
        Forward Pass the model
        Args:
            x_source (torch.Tensor): the input data tensor.
                x_source.shape is (batch, seq_size)
            x_lengths (torch.Tensor): a vector of lengths for each item in the batch
        Returns:
            a tuple: x_unpacked (torch.Tensor), x_birnn_h (torch.Tensor)
                x_unpacked.shape = (batch, seq_size, rnn_hidden_size * 2)
                x_birnn_h.shape = (batch, rnn_hidden_size * 2)
        """
        x_embedded = self.source_embedding(x_source)
        x_lengths = x_lengths.detach().cpu().numpy()

        # create PackedSequence;
        # x_packed.data.shape=(number_items, embedding_size)
        x_packed = pack_padded_sequence(x_embedded, x_lengths, batch_first=True)
        
        # x_birnn_h.shape = (num_rnn, batch_size, feature_size)
        x_birnn_out, x_birnn_h = self.birnn(x_packed)
        # permute to (batch_size, num_rnn, feature_size)
        x_birnn_h = x_birnn_h.permute(1,0,2)
        
        # flatten features; reshape to (batch_size, num_rnn * feature_size)
        #  (recall: -1 takes the remaining positions, 
        #           flattening the two RNN hidden vectors into 1)
        x_birnn_h = x_birnn_h.contiguous().view(x_birnn_h.size(0), -1)
        
        x_unpacked, _ = pad_packed_sequence(x_birnn_out, batch_first=True)
        return x_unpacked, x_birnn_h

## Decoder

In [ ]:
def verbose_attention(encoder_state_vectors, query_vector):
    """A descriptive version of the neural attention mechanism 
    
    Args:
        encoder_state_vectors (torch.Tensor): 3dim tensor from bi-GRU in encoder
        query_vector (torch.Tensor): hidden state in decoder GRU
    """
    batch_size, num_vectors, vector_size = encoder_state_vectors.size()
    
    vector_scores = \
        torch.sum(encoder_state_vectors * query_vector.view(batch_size, 1, vector_size), dim=2)
    
    vector_probabilities = torch.softmax(vector_scores, dim=1)
    
    weighted_vectors = \
        encoder_state_vectors * vector_probabilities.view(batch_size, num_vectors, 1)
    
    context_vectors = torch.sum(weighted_vectors, dim=1)
    return context_vectors, vector_probabilities, vector_scores

def terse_attention(encoder_state_vectors, query_vector):
    """A shorter and more optimized version of the neural attention mechanism
    
    Args:
        encoder_state_vectors (torch.Tensor): 3dim tensor from bi-GRU in encoder
        query_vector (torch.Tensor): hidden state
    """
    vector_scores = torch.matmul(encoder_state_vectors, query_vector.unsqueeze(dim=2)).squeeze()
    
    vector_probabilities = torch.softmax(encoder_state_vectors, dim=-1)
    context_vectors = torch.matmul(encoder_state_vectors.transpose(-2, -1),
                                  vector_probabilities.unsqueeze(dim=2)).squeeze()
    
    return context_vectors, vector_probabilities

class NMTDecoder(nn.Module):
    def __init__(self, num_embeddings, embedding_size, rnn_hidden_size, bos_index):
        super(NMTDecoder, self).__init__()
        
        self._rnn_hidden_size = rnn_hidden_size
        self.target_embedding = nn.Embedding(num_embeddings=num_embeddings,
                                            embedding_dim=embedding_size,
                                            padding_idx=0)
        
        self.gru_cell = nn.GRUCell(embedding_size + rnn_hidden_size, rnn_hidden_size)
        self.hidden_map = nn.Linear(rnn_hidden_size, rnn_hidden_size)
        
        self.classifier = nn.Linear(rnn_hidden_size*2, num_embeddings)
        self.bos_index = bos_index
        
    def _init_indices(self, batch_size):
        """returns the BOS index vector"""
        return torch.ones(batch_size, dtype=torch.int64) * self.bos_index
            
    def _init_context_vectors(self, batch_size):
        """returns a zeros vector for initializing the context"""
        return torch.zeros(batch_size, self._rnn_hidden_size)
    
    def forward(self, encoder_state, initial_hidden_state, target_sequence):
        """The forward pass of the model
        
        Args:
            encoder_state (torch.Tensor): the output of the NMTEncoder
            initial_hidden_state (torch.Tensor): The last hidden state in the  NMTEncoder
            target_sequence (torch.Tensor): the target text data tensor
        Returns:
            output_vectors (torch.Tensor): prediction vectors at each output step
        """

        target_sequence = target_sequence.permute(1,0)
        
        h_t = self.hidden_map(initial_hidden_state)
        
        batch_size = encoder_state.size(0)
        
        # initialize context vector
        context_vectors = self._init_context_vectors(batch_size)
        y_t_index = self._init_indices(batch_size)
        
        h_t = h_t.to(encoder_state.device)
        y_t_index = y_t_index.to(encoder_state.device)
        context_vectors = context_vectors.to(encoder_state.device)
        
        output_vectors = []
        self._cached_p_attn = []
        self._cached_ht = []
        self._cached_decoder_state = encoder_state.cpu().detach().numpy()
        
        output_sequence_size = target_sequence.size(0)
        for i in range(output_sequence_size):

            y_t_index = target_sequence[i]
            
            # decoding the vectors
            # 1. embed word and concat with previous context
            y_input_vector = self.target_embedding(y_t_index)
            rnn_input = torch.cat([y_input_vector, context_vectors], dim=1)
            
            # 2. make a GRU step, getting a new hidden vector
            h_t = self.gru_cell(rnn_input, h_t)
            self._cached_ht.append(h_t.cpu().detach().numpy())
            
            # 3. use current vector to attend to encoder state
            context_vectors, p_attn, _ = verbose_attention(encoder_state_vectors=encoder_state,
                                                           query_vector = h_t)
            
            # cache the attention probabilities for visualization
            self._cached_p_attn.append(p_attn.cpu().detach().numpy())
            
            # 4 use current hidden and context vectors
            # to make a prediction for the next word
            prediction_vector = torch.cat((context_vectors, h_t), dim=1)
            score_for_y_t_index = self.classifier(prediction_vector)
            
            # collect the prediction scores
            output_vectors.append(score_for_y_t_index)

        output_vectors = torch.stack(output_vectors).permute(1, 0, 2)
        
        return output_vectors

## NMT Model

In [ ]:
class NMTModel(nn.Module):
    def __init__(self, source_vocab_size, source_embedding_size, target_vocab_size,
                target_embedding_size, encoding_size, target_bos_index):
        """
        Args:
            source_vocab_size (int): number of unique words in source language
            source_embedding_size (int): size of the source embedding vectors
            target_vocab_size (int): number of unique words in target language
            target_embedding_size (int): size of the target embedding vectors
            encoding_size (int): the size of the encoder RNN.  
        """
        super(NMTModel, self).__init__()
        
        self.encoder = NMTEncoder(num_embeddings= source_vocab_size,
                                 embedding_size=source_embedding_size,
                                 rnn_hidden_size=encoding_size)
        
        decoding_size = encoding_size * 2
        
        self.decoder = NMTDecoder(num_embeddings= target_vocab_size,
                                 embedding_size= target_embedding_size,
                                 rnn_hidden_size= decoding_size,
                                 bos_index= target_bos_index)
        
    def forward(self, x_source, x_source_lengths, target_sequence):
        """
        The forward pass of the model
        
        Args:
            x_source (torch.Tensor): the source text data tensor. 
                x_source.shape should be (batch, vectorizer.max_source_length)
            x_source_lengths torch.Tensor): the length of the sequences in x_source 
            target_sequence (torch.Tensor): the target text data tensor
        Returns:
            decoded_states (torch.Tensor): prediction vectors at each output step
        """
        encoder_state, final_hidden_states = self.encoder(x_source,
                                                         x_source_lengths)
        decoded_states = self.decoder(encoder_state, final_hidden_states,
                                     target_sequence)
        
        return decoded_states

## Utility functions

In [ ]:
def compute_accuracy(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.max(dim=1)
    
    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()
    
    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100

def normalize_sizes(y_pred, y_true):
    """Normalize tensor sizes
    
    Args:
        y_pred (torch.Tensor): the output of the model
            If a 3-dimensional tensor, reshapes to a matrix
        y_true (torch.Tensor): the target predictions
            If a matrix, reshapes to be a vector
    """
    if len(y_pred.size()) == 3:
        y_pred = y_pred.contiguous().view(-1, y_pred.size(2))
    if len(y_true.size()) == 2:
        y_true = y_true.contiguous().view(-1)
    return y_pred, y_true

def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)


def update_train_state(args, model, train_state):
    """
    Update model and early stopping
    """
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])

    # save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
         
        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

## Training Routine

In [ ]:
args = Namespace(
    # Data information
    frequency_cutoff = 25,
    text_csv = '/content/drive/My Drive/Colab Notebooks/PoetryGenerator/data/poet_dataframe_sentences.csv',
    model_filename = '/content/drive/My Drive/Colab Notebooks/PoetryGenerator/model/poet_GRU_model_state.pth',
    # Model HyperParameters
    source_embedding_size=100,
    target_embedding_size=100,
    encoding_size=64,
    # Training HyperParameters
    batch_size = 128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    momentum=0.1,
    num_epochs=100,
    seed=1337,
    cuda=True,
    dropout=0.1
)

def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_filename
    }

In [ ]:
train_state = make_train_state(args)

if torch.cuda.is_available() and args.cuda:
  args.cuda = True
else:
  args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Device available ", args.device)

# dataset object
dataset = NMTDataset.load_dataset_and_make_vectorizer(args.text_csv)

# vectorizer
vectorizer = dataset.get_vectorizer()

# classifier
model = NMTModel(source_vocab_size= len(vectorizer.source_vocab),
                target_vocab_size= len(vectorizer.target_vocab),
                source_embedding_size = args.source_embedding_size,
                target_embedding_size= args.target_embedding_size,
                encoding_size= args.encoding_size,
                target_bos_index= vectorizer.target_vocab.begin_seq_index)
model.to(args.device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

Device available  cuda


In [ ]:
mask_index = vectorizer.target_vocab.mask_index
epoch_bar = tqdm(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=0, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=0, 
                        leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_nmt_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        model.train()
        
        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------    
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = model(batch_dict['x_source'], 
                           batch_dict['x_source_length'], 
                           batch_dict['x_target'])

            # step 3. compute the loss
            loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)


            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the  running loss and running accuracy
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss,
                                  acc=running_acc,
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_nmt_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        model.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # compute the output
            y_pred = model(batch_dict['x_source'], 
                           batch_dict['x_source_length'], 
                           batch_dict['x_target'])

            # step 3. compute the loss
            loss = sequence_loss(y_pred, batch_dict['y_target'], mask_index)

            # compute the  running loss and running accuracy
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'], mask_index)
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            # Update bar
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=model, 
                                         train_state=train_state)
        
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
except KeyboardInterrupt:
    print("Exiting loop")

## Evaluate

In [ ]:
def get_source_sentence(vectorizer, batch_dict, index):
    indices = batch_dict['x_source'][index].cpu().data.numpy()
    vocab = vectorizer.source_vocab
    return sentence_from_indices(indices, vocab)

def get_true_sentence(vectorizer, batch_dict, index):
    return sentence_from_indices(batch_dict['y_target'].cpu().data.numpy()[index], vectorizer.target_vocab)
    
def get_sampled_sentence(vectorizer, batch_dict, index):
    y_pred = model(x_source=batch_dict['x_source'], 
                   x_source_lengths=batch_dict['x_source_length'], 
                   target_sequence=batch_dict['x_target'])
    return sentence_from_indices(torch.max(y_pred, dim=2)[1].cpu().data.numpy()[index], vectorizer.target_vocab)

def get_all_sentences(vectorizer, batch_dict, index):
    return {"source": get_source_sentence(vectorizer, batch_dict, index), 
            "truth": get_true_sentence(vectorizer, batch_dict, index), 
            "sampled": get_sampled_sentence(vectorizer, batch_dict, index)}
    
def sentence_from_indices(indices, vocab, strict=True):
    ignore_indices = set([vocab.mask_index, vocab.begin_seq_index, vocab.end_seq_index])
    out = []
    for index in indices:
        if index == vocab.begin_seq_index and strict:
            continue
        elif index == vocab.end_seq_index and strict:
            return " ".join(out)
        else:
            out.append(vocab.lookup_index(index))
    return " ".join(out)


dataset.set_split('test')
batch_generator = generate_nmt_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)

test_sample = '/content/drive/MyDrive/Colab Notebooks/PoetryGenerator/data/test_sample.txt'
val_sample = '/content/drive/MyDrive/Colab Notebooks/PoetryGenerator/data/val_sample.txt'

for batch_dict in batch_generator:
    results = get_all_sentences(vectorizer, batch_dict, 0)
    print('-'*100)
    print("SOURCE : ", results['source'])
    print("SAMPLED: ", results['sampled'])
    print("TRUTH: ",results['truth'])
    
    with open(test_sample, 'a') as fp:
        fp.write("-"*100)
        fp.write("\n")
        fp.write("SOURCE : {}".format(results['source']))
        fp.write("\n")
        fp.write("SAMPLED : {}".format(results['sampled'])) 
        fp.write("\n")
        fp.write("TRUTH : {}".format(results['truth']))
        fp.write("\n")

    fp.close()


----------------------------------------------------------------------------------------------------
SOURCE :  i the bronze <UNK> grant riding a bronze horse in <UNK> <UNK> <UNK> in the sun by day when the motor <UNK> <UNK> by in long <UNK> <UNK> somewhere to keep <UNK> for <UNK> and <UNK> and <UNK> and <UNK> though in the dusk and <UNK> when high waves are <UNK> on the <UNK> of the <UNK> along the lake shore near by i have seen the <UNK> dare the <UNK> come <UNK> and make to ride his bronze horse out into the hoofs and <UNK> of the storm . 
SAMPLED:  and i did it the the the wind , . it wind wind simple out 
TRUTH:  ii i cross <UNK> <UNK> on a winter night when the snow is falling . 
----------------------------------------------------------------------------------------------------
SOURCE :  thereto do thou , great goddess , queen of beauty , mother of love , and of all world s delight , without whose sovereign grace and kindly duty nothing on earth seems fair to fleshly sight , do t